In [33]:
# Import the data from csv file and explore the data

import os
import sys
import pandas as pd
import logging
from google.cloud import bigquery
from google.oauth2 import service_account
from hashlib import md5
from typing import List
import json

# **** SETUP ****

# change to match your filesystem
DATA_DIR = "./data/"
DEFAULT_RECEIPTS_FILE = os.path.join(DATA_DIR, "World_Energy_Consumption.csv")
PROJECT_NAME = "emissions-team-project"
DATASET_NAME = "emissions"

# read the csv for the raw data
df = pd.read_csv(DEFAULT_RECEIPTS_FILE)

display(df.head(n=10))



,iso_code,country,year,coal_prod_change_pct,coal_prod_change_twh,gas_prod_change_pct,gas_prod_change_twh,oil_prod_change_pct,oil_prod_change_twh,energy_cons_change_pct,...,solar_elec_per_capita,solar_energy_per_capita,gdp,wind_share_elec,wind_cons_change_pct,wind_share_energy,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_energy_per_capita
0,AFG,Afghanistan,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1901,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,1902,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,1903,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,1904,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AFG,Afghanistan,1905,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AFG,Afghanistan,1906,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AFG,Afghanistan,1907,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AFG,Afghanistan,1908,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AFG,Afghanistan,1909,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# **** SETUP LOGGING ****
# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level


# **** BIGQUERY CLIENT ****
logger.debug(f"Creating bigquery client")

key_path = "/Users/Ruben/Desktop/google_cred/.cred/emissions-team-project-10e7e81d45bb.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

logger.info(f"Setup Completed")

[DEBUG][2023-01-18 13:53:54,116][4263563694:0013] : Creating bigquery client
[INFO ][2023-01-18 13:53:54,129][4263563694:0021] : Setup Completed


In [35]:
# **** BIQUERY SCHEMA TABLE SETUP ****

FACTS_TABLE_METADATA = {
    'fct_gdp': {
        'table_name': 'fct_gdp',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('country_code_year', 'string', mode='REQUIRED'),
            bigquery.SchemaField('country_code', 'string', mode='REQUIRED'),
            bigquery.SchemaField('year', 'int64', mode='NULLABLE'),
            bigquery.SchemaField('gdp', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('population', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('created_at', 'timestamp', mode='NULLABLE'),
            bigquery.SchemaField('modified_at', 'timestamp', mode='NULLABLE'),
        ],
    },
    'fct_consump': {
        'table_name': 'fct_consump',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('country_code_year', 'string', mode='REQUIRED'),
            bigquery.SchemaField('country_code', 'string', mode='REQUIRED'),
            bigquery.SchemaField('year', 'int64', mode='NULLABLE'),
            bigquery.SchemaField('biofuel_consumption', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('coal_consumption', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('fossil_fuel_consumption', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('gas_consumption', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('hydro_consumption', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('low_carbon_consumption', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('oil_consumption', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('other_renewable_consumption', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('primary_energy_consumption', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('renewables_consumption', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('solar_consumption', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('wind_consumption', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('created_at', 'timestamp', mode='NULLABLE'),
            bigquery.SchemaField('modified_at', 'timestamp', mode='NULLABLE'),
        ],
    },
    
}

In [36]:
# **** BIGQUERY DATASET CREATION ****

dataset_id = f"{PROJECT_NAME}.{DATASET_NAME}"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok=True)

logger.info(f"Created emissions dataset: {dataset.full_dataset_id}")

[INFO ][2023-01-18 13:54:16,233][2746255190:0008] : Created emissions dataset: emissions-team-project:emissions


In [27]:
# create the gdf dataframe in order to prep for fct_gdp table creation

# extract necessary columns from raw csv file
gdp_df = df
gdp_cols = ['iso_code', 'country', 'year', 'gdp', 'population']
gdp_df = gdp_df[gdp_cols]


# rename columns to standardize across all the tables
gdp_df = gdp_df.rename(columns={
    'iso_code': 'country_code', 
    'country':'country_name', 
    })

# create composite key column for the fct_gdp table
def composite_key(row):
    comp_key = f"{row.country_code}{row.year}"
    return comp_key

gdp_df.insert(0, 'country_code_year', gdp_df.apply(composite_key, axis=1))

gdp_df.drop_duplicates(inplace=True)

# create created_at and modified_at columns
gdp_df['created_at'] = pd.Timestamp.now()
gdp_df['modified_at'] = None

gdp_df.set_index('country_code_year', inplace=True)

display(gdp_df)
gdp_df.dtypes




,country_code,country_name,year,gdp,population,created_at,modified_at
country_code_year,,,,,,,
AFG1900,AFG,Afghanistan,1900,NaN,5021241.0,2023-01-18 12:30:20.822002,None
AFG1901,AFG,Afghanistan,1901,NaN,5053439.0,2023-01-18 12:30:20.822002,None
AFG1902,AFG,Afghanistan,1902,NaN,5085403.0,2023-01-18 12:30:20.822002,None
AFG1903,AFG,Afghanistan,1903,NaN,5118005.0,2023-01-18 12:30:20.822002,None
AFG1904,AFG,Afghanistan,1904,NaN,5150814.0,2023-01-18 12:30:20.822002,None
...,...,...,...,...,...,...,...
ZWE2015,ZWE,Zimbabwe,2015,2.503057e+10,13815000.0,2023-01-18 12:30:20.822002,None
ZWE2016,ZWE,Zimbabwe,2016,2.515176e+10,14030000.0,2023-01-18 12:30:20.822002,None
ZWE2017,ZWE,Zimbabwe,2017,NaN,14237000.0,2023-01-18 12:30:20.822002,None


country_code            object
country_name            object
year                     int64
gdp                    float64
population             float64
created_at      datetime64[ns]
modified_at             object
dtype: object

In [28]:
# create the consumption dataframe to prep for fct_consump table

consump_df = df
consump_cols = [
    'iso_code', 
    'country', 
    'year', 
    'biofuel_consumption', 
    'coal_consumption',
    'fossil_fuel_consumption',
    'gas_consumption',
    'hydro_consumption',
    'low_carbon_consumption',
    'nuclear_consumption',
    'oil_consumption',
    'other_renewable_consumption',
    'primary_energy_consumption',
    'renewables_consumption',
    'solar_consumption',
    'wind_consumption']

# isolate for necessary columns
consump_df = consump_df[consump_cols]

# rename columns to standardized names
consump_df = consump_df.rename(columns={
    'iso_code': 'country_code', 
    'country':'country_name', 
    })

consump_df.drop_duplicates(inplace=True)

# create composite key column for the fct_consump table
def composite_key(row):
    comp_key = f"{row.country_code}{row.year}"
    return comp_key

consump_df.insert(0, 'country_code_year', consump_df.apply(composite_key, axis=1))

# create created_at and modified_at columns
consump_df['created_at'] = pd.Timestamp.now()
consump_df['modified_at'] = None

consump_df.set_index('country_code_year', inplace=True)

display(consump_df)

consump_df.dtypes

,country_code,country_name,year,biofuel_consumption,coal_consumption,fossil_fuel_consumption,gas_consumption,hydro_consumption,low_carbon_consumption,nuclear_consumption,oil_consumption,other_renewable_consumption,primary_energy_consumption,renewables_consumption,solar_consumption,wind_consumption,created_at,modified_at
country_code_year,,,,,,,,,,,,,,,,,,
AFG1900,AFG,Afghanistan,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-18 12:30:25.494009,None
AFG1901,AFG,Afghanistan,1901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-18 12:30:25.494009,None
AFG1902,AFG,Afghanistan,1902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-18 12:30:25.494009,None
AFG1903,AFG,Afghanistan,1903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-18 12:30:25.494009,None
AFG1904,AFG,Afghanistan,1904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-18 12:30:25.494009,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWE2015,ZWE,Zimbabwe,2015,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.642,NaN,NaN,NaN,2023-01-18 12:30:25.494009,None
ZWE2016,ZWE,Zimbabwe,2016,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.500,NaN,NaN,NaN,2023-01-18 12:30:25.494009,None
ZWE2017,ZWE,Zimbabwe,2017,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-18 12:30:25.494009,None


country_code                           object
country_name                           object
year                                    int64
biofuel_consumption                   float64
coal_consumption                      float64
fossil_fuel_consumption               float64
gas_consumption                       float64
hydro_consumption                     float64
low_carbon_consumption                float64
nuclear_consumption                   float64
oil_consumption                       float64
other_renewable_consumption           float64
primary_energy_consumption            float64
renewables_consumption                float64
solar_consumption                     float64
wind_consumption                      float64
created_at                     datetime64[ns]
modified_at                            object
dtype: object

In [29]:
# define the load table function

def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: List[bigquery.SchemaField], 
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_TRUNCATE'
    ) -> None:
    """load dataframe into bigquery table

    Args:
        df (pd.DataFrame): dataframe to load
        client (bigquery.Client): bigquery client
        table_name (str): full table name including project and dataset id
        schema (List[bigquery.SchemaField]): table schema with data types
        create_disposition (str, optional): create table disposition. Defaults to 'CREATE_IF_NEEDED'.
        write_disposition (str, optional): overwrite table disposition. Defaults to 'WRITE_TRUNCATE'.
    """
    # *** run some checks ***
    # test table name to be full table name including project and dataset name. It must contain to dots
    assert len(table_name.split('.')) == 3, f"Table name must be a full bigquery table name including project and dataset id: '{table_name}'"
    # setup bigquery load job:
    #  create table if needed, replace rows, define the table schema
    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    logger.info(f"loading table: '{table_name}'")
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result()        # wait for the job to finish
    # get the resulting table
    table = client.get_table(table_name)
    logger.info(f"loaded {table.num_rows} rows into {table.full_table_id}")

In [30]:
# Load gdp fact table

# get table name and schema from FACTS_TABLE_METADATA config param
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{FACTS_TABLE_METADATA['fct_gdp']['table_name']}"
schema = FACTS_TABLE_METADATA['fct_gdp']['schema']
# load dataframe
load_table(gdp_df, client, table_name, schema)

logger.info(f"loaded gdp facts")

[INFO ][2023-01-18 12:30:35,614][2444475212:0031] : loading table: 'emissions-team-project.emissions.fct_gdp'


Forbidden: 403 POST https://bigquery.googleapis.com/upload/bigquery/v2/projects/deb-01-371820/jobs?uploadType=multipart: Access Denied: Dataset emissions-team-project:emissions: Permission bigquery.tables.create denied on dataset emissions-team-project:emissions (or it may not exist).

In [14]:
# Load consumption fact table

# get table name and schema from FACTS_TABLE_METADATA config param
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{FACTS_TABLE_METADATA['fct_consump']['table_name']}"
schema = FACTS_TABLE_METADATA['fct_consump']['schema']
# load dataframe
load_table(consump_df, client, table_name, schema)

logger.info(f"loaded consumption facts")

[INFO ][2023-01-18 12:19:38,231][2444475212:0031] : loading table: 'emissions-team-project.emissions.fct_consump'


Forbidden: 403 POST https://bigquery.googleapis.com/upload/bigquery/v2/projects/deb-01-371820/jobs?uploadType=multipart: Access Denied: Dataset emissions-team-project:emissions: Permission bigquery.tables.create denied on dataset emissions-team-project:emissions (or it may not exist).